In [19]:
%mkdir dataset
# Accuracy for Comments (Fine-tuned): 0.827

mkdir: cannot create directory ‘dataset’: File exists


# load datasets


In [20]:
import pandas as pd
#loading the id and the LABELS
sentiment_pd = pd.read_csv("/content/dataset/sentiment_labels.txt",
                  sep="|",
                  header=0,
                  names=['phrase_id', 's_label'],
                  on_bad_lines='skip')

#loading the id and the PHRASES
phrases_pd = pd.read_csv("/content/dataset/dictionary.txt",
                         sep="|",
                         header=None,
                         on_bad_lines='skip',
                         names=['phrase', 'phrase_id'])
v = pd.read_csv("/content/dataset/dictionary.txt", sep="|", header=0)

In [21]:
comments = pd.read_csv("/content/dataset/Reddit_Data.csv")


#Discretizzare labels

In [22]:
categories = { 0.34:'negative', 0.65:'neutral', 1:'positive'}
categories_comments = { -1 :'negative', 0:'neutral', 1:'positive'}
thresholds = list(categories.keys())
thresholds_c = list(categories_comments.keys())

sentiment_pd['s_label_disc'] = sentiment_pd['s_label'].apply(lambda x: next(categories[t] for t in thresholds if x <= t), 'OUTLIER')

comments['category_disc'] = comments['category'].apply(lambda x: next(categories_comments[t] for t in thresholds_c if x==t))
# comments['category_disc'] = comments['category'].apply(lambda x: next(categories[t] for t in thresholds if x<=t))

In [23]:
reviews = pd.DataFrame({'phrase_id': sentiment_pd['phrase_id'], 'phrase': phrases_pd['phrase'], 's_label_disc':sentiment_pd['s_label_disc'],  's_label': sentiment_pd['s_label']})
reviews['phrase'].loc[reviews['s_label'].idxmax()]

"'s not Little Nicky . And for many of us , that 's good enough"

# **data cleaning**

#EMPTY DATA cleaning

In [24]:
#Remove NON STRING ELEMENTS and EMPTY STRINGS
comments = comments[comments['clean_comment'].apply(lambda x: isinstance(x,str) and len(x) > 0)]
#lower strings
comments['clean_comment'] = comments['clean_comment'].apply(lambda x: x.lower())

#Remove NON STRING ELEMENTS and EMPTY STRINGS
reviews = reviews[reviews['phrase'].apply(lambda x: isinstance(x,str) and len(x) > 0)]
#lower strings
reviews['phrase'] = reviews['phrase'].apply(lambda x: x.lower())

# removing stop words

In [25]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = list(stopwords.words('english'))
#stop_words_df = pd.DataFrame(data=stop_words, columns=['STOP_WORD'])
#Removing the stopwords in x
comments['clean_comment'] = comments['clean_comment'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

#Removing the stopwords in x
reviews['phrase'] = reviews['phrase'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [26]:
#punctuation removal nuetral
import string
comments['clean_comment'] = comments['clean_comment'].apply(lambda x: ' '.join([word for word in x.split() if word not in string.punctuation]))

#remove empty raws
comments = comments[comments['clean_comment'].apply(lambda x: isinstance(x,str) and len(x) > 0)]

#punctuation removal nuetral
import string
reviews['phrase'] = reviews['phrase'].apply(lambda x: ' '.join([word for word in x.split() if word not in string.punctuation]))

#remove empty raws
reviews = reviews[reviews['phrase'].apply(lambda x: isinstance(x,str) and len(x) > 0)]

#neutral is cleaned

In [27]:
import nltk
from nltk.corpus import stopwords

# Download NLTK resources if not already installed
nltk.download('punkt')
nltk.download('stopwords')

def remove_punctuation_stopwords(text):
  # Tokenize the text
  tokens = nltk.word_tokenize(text)
  # Filter out punctuation and stopwords
  stop_words = set(stopwords.words('english'))  # Download stopwords for your language
  filtered_tokens = [word.lower() for word in tokens if word.isalnum() and word not in stop_words]
  return ' '.join(filtered_tokens)

reviews['phrase'] = reviews['phrase'].apply(remove_punctuation_stopwords)
comments['clean_comment'] = comments['clean_comment'].apply(remove_punctuation_stopwords)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [28]:
reviews['phrase'] = reviews['phrase'].apply(lambda x: ' '.join([char for char in x.split() if not char.isdigit()]))

#remove empty raws
reviews = reviews[reviews['phrase'].apply(lambda x: isinstance(x,str) and len(x) > 0)]

comments['clean_comment'] = comments['clean_comment'].apply(lambda x: ' '.join([char for char in x.split() if not char.isdigit()]))

#remove empty raws
comments = comments[comments['clean_comment'].apply(lambda x: isinstance(x,str) and len(x) > 0)]

# **Learning**

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Assuming you have two datasets: reviews and comments

# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=10000)

# Fit and transform the training data (reviews)
X_train_tfidf = tfidf_vectorizer.fit_transform(reviews['phrase'])
y_train_reviews = reviews['s_label_disc']

# Initialize logistic regression classifier
classifier = LogisticRegression(max_iter=10000)

# Train the classifier on reviews dataset
classifier.fit(X_train_tfidf, y_train_reviews)

# Evaluate the classifier on comments dataset


# --- Fine-tuning on Comments (Optional) ---

# Split comments for training and testing
X_train_comments, X_test_comments, y_train_comments, y_test_comments = train_test_split(comments['clean_comment'],
                                                                                        comments['category_disc'],
                                                                                        test_size=0.2,
                                                                                        random_state=42)

# Transform comments training data using the fitted vectorizer
X_train_tfidf_comments = tfidf_vectorizer.transform(X_train_comments)

# Fine-tune the classifier on comments training data
classifier.fit(X_train_tfidf_comments, y_train_comments)  # Re-train

# Evaluate the fine-tuned classifier on comments testing data
X_test_tfidf_comments = tfidf_vectorizer.transform(X_test_comments)
y_pred_comments = classifier.predict(X_test_tfidf_comments)

# Print classification report
print("Classification Report for Comments (Fine-tuned):")
print(classification_report(y_test_comments, y_pred_comments))

# Print Accuracy
acc = accuracy_score(y_test_comments, y_pred_comments)
print("Accuracy for Comments (Fine-tuned): {:0.3f}".format(acc))



Classification Report for Comments (Fine-tuned):
              precision    recall  f1-score   support

    negative       0.82      0.58      0.68      1636
     neutral       0.79      0.96      0.87      2537
    positive       0.87      0.85      0.86      3202

    accuracy                           0.83      7375
   macro avg       0.83      0.80      0.80      7375
weighted avg       0.83      0.83      0.82      7375

Accuracy for Comments (Fine-tuned): 0.827


In [30]:
#Testing
test_phrases1 = ['test phrasing is good ', 'i don wanna go', 'that is some big Bullshit man']
test_phrases = [
  "the book has 300 pages",
  "the coffee is on the table",
  "This movie was absolutely fantastic! I highly recommend it.",
  "The service at this restaurant was terrible. I will never go back.",
  "I'm so happy with my new phone! It's everything I wanted and more.",
  "This product is a complete waste of money. It doesn't work at all.",
  "I had a wonderful time at the concert last night. The music was amazing!",
  "The weather today is awful. It's raining cats and dogs!",
  "I'm feeling very optimistic about the future. Things are looking up!",
  "I'm so disappointed with the outcome of the election. It's a real shame.",
  "This book is a real page-turner. I couldn't put it down!",
  "I'm bored out of my mind. There's nothing to do!",
]


pd.DataFrame({'LABEL': classifier.predict(tfidf_vectorizer.transform(test_phrases)), 'FRASE': test_phrases})
# pd.DataFrame({'LABEL': clas.predict(tfidf_vectorizer.transform(test_phrases1)), 'FRASE': test_phrases1})

,LABEL,FRASE
0,neutral,the book has 300 pages
1,neutral,the coffee is on the table
2,positive,This movie was absolutely fantastic! I highly ...
3,negative,The service at this restaurant was terrible. I...
4,positive,I'm so happy with my new phone! It's everythin...
5,negative,This product is a complete waste of money. It ...
6,positive,I had a wonderful time at the concert last nig...
7,negative,The weather today is awful. It's raining cats ...
8,neutral,I'm feeling very optimistic about the future. ...
9,negative,I'm so disappointed with the outcome of the el...
